## 第九週作業 04/15: 打造你專屬的 AI Agent
📌 臺灣師範大學 | 613K0019C | AI所碩一 | 劉思廷

🔖 設計模式: **Planning 模式**

🔖 原始任務: **Prompt 生成器**

✔️ Two-stage CoT 流程:

  1. 第一階段 (思考階段): 請 LLM 根據使用者簡單的 Prompt 要求，生成適用這個任務的完整敘述
  2. 第二階段 (生成階段): 請根據第一階段生成的敘述方式改寫成可以最佳化 LLM 能力的 Prompt

In [1]:
!pip install gradio
!pip install aisuite[all]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.9/863.9 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3

In [2]:
""" 固定四行引用套件 """
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os # 進行與作業系統相關的操作，例如檔案與目錄的處理
import aisuite as ai # 提供各種人工智慧應用相關的工具與功能，簡化AI開發流程
from google.colab import userdata # 用來存取使用者上傳的私人資料 (API 金鑰)

In [3]:
""" 使用 Groq 的 API 金鑰 """
api_key = userdata.get('Groq') # 從使用者上傳的私人資料中取得 'Groq' 的 API 金鑰
os.environ['GROQ_API_KEY']=api_key # 將取得的 API 金鑰設定到環境變數 'GROQ_API_KEY' 中，供後續程式使用

In [4]:
""" 設定 planner & writer 的 LLM 模型 """
provider_planner = "groq" # 設定規劃器 (planner) 的服務提供者為 Groq
model_planner = "gemma2-9b-it" # 指定規劃器使用的模型為 Gemma2-9B-IT

provider_writer = "groq" # 設定撰寫器 (writer) 的服務提供者為 Groq
model_writer = "llama3-70b-8192" # 指定撰寫器使用的模型為 Llama3-70B-8192

In [5]:
# Planner 的 System Prompt
system_planner = ''' 請用台灣習慣的中文(繁體中文)回應:
假如你是一位資深的Prompt設計規劃師，擅長理解使用者需求並將其轉化成成適合大型語言模型（LLM）處理的明確指示。你的任務是：
1. 仔細分析使用者提供的簡單需求或任務描述。
2. 主動推測使用者可能的真實目標，包括可能隱含的背景或動機。
3. 以清楚、完整、具體的方式，重寫成一段「完整任務敘述」。
  - 這段敘述需包含：
    - 明確的任務目標
    - 任務需要達成的標準（例如語氣、格式、長度）
    - 任何已知的背景資訊（如果可以推測）
    - 任何輸出時應該遵守的特別要求（如角色扮演、風格偏好）
4. 採用條列式或分段敘述，避免單一長段落。 '''

# Writer 的 System Prompt
system_writer = ''' 請用台灣習慣的中文(繁體中文)回應:
你是一位經驗豐富的Prompt工程師，擅長將使用者目標轉換成清晰、有效、能最大化大型語言模型（LLM）能力的指令。你的任務是：
1. 根據第一階段生成的「完整任務敘述」，撰寫一個最適合用於LLM的Prompt。
2. 這個Prompt應該包含：
   - 明確的角色設定（例如：「你是一名專業...」、「你是...」）
   - 清楚的任務描述
   - 必要的格式要求（如列點、表格、段落、第一人稱/第三人稱視角）
   - 特別的語氣或風格要求（如正式、幽默、親切、冷靜等）
   - 若適合，可以要求模型在回覆前思考或列出步驟（即CoT技巧）
3. 採用直覺易懂、容易讓LLM正確理解並執行的語句結構。
請保持指令簡潔、具體、正面，引導模型朝著最佳表現前進。 '''

In [6]:
""" 定義 reply 函式: 用來根據輸入的 system prompt 和 user prompt，呼叫指定的 AI 模型產生回覆 """
def reply(system, prompt, provider ,model):

    client = ai.Client() # 建立一個 aisuite 的 Client 實例，用來與 AI 模型進行對話

    messages = [
        {"role": "system", "content": system}, # 設定系統提示 (system prompt)，指導模型行為
        {"role": "user", "content": prompt}  # 設定使用者輸入 (user prompt)，作為模型的輸入內容
    ]

    response = client.chat.completions.create(model=f"{provider}:{model}", messages=messages)
    # 呼叫 chat API，使用指定的 provider 和 model，傳入訊息並取得回應

    return response.choices[0].message.content # 回傳模型產生的文字內容作為回覆

In [7]:
""" 定義 prompt_gen 函式: 用來根據使用者的初始需求，生成經過優化的提示詞 (Prompt) """
def prompt_gen(prompt):
    # Step 1: CoT - 使用規劃器 (planner) 依據使用者需求，生成更具結構性的敘述
    planning_prompt = f"使用者提供的簡單需求或任務描述：{prompt}"
    statement = reply(system_planner, planning_prompt, provider=provider_planner, model=model_planner)

    # Step 2: 進一步將結構化敘述轉換成能最佳化 LLM 能力的最終提示詞
    generation_prompt = f"這是經過模型優化的使用者要求敘述：\n{statement}\n\n請幫我生成最終能夠最佳化LLM能力的Prompt"
    final_prompt = reply(system_writer, generation_prompt, provider=provider_planner, model=model_planner)

    return statement, final_prompt # 回傳中間生成的結構化敘述與最終優化的提示詞

In [ ]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("### 🛠️ Prompt生成器 Demo - 用兩階段CoT幫你打造完美Prompt")
    gr.Markdown("這個工具可以根據你簡單輸入的任務需求，自動分兩步驟思考並生成最適合LLM執行的高品質Prompt")
    user_input = gr.Textbox(label="請盡可能描述你想要完成的任務，可以不需要是流暢的敘述")
    btn = gr.Button("生成完整Prompt")

    with gr.Row():
        out1 = gr.Textbox(label="🔍 第一階段：任務目標與完整敘述")
        out2 = gr.Textbox(label="✍️ 第二階段：最佳化後的LLM Prompt")

    btn.click(prompt_gen, inputs=[user_input], outputs=[out1, out2])

In [ ]:
demo.launch(share=True, debug=True)